In [1]:
pip install ib_insync

In [5]:
from ib_insync import *

# Utilize an instance of IB
ib = IB()

# Connect to the IB server using await in front of the connectAsync method
try:
    await ib.connectAsync('127.0.0.1', 7497, clientId=2)
    print("Connected to Interactive Brokers")
except RuntimeError as e:
    print(f"Runtime Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
# from ib_insync import *

# ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=2)

Connected to Interactive Brokers


In [6]:
contract = Future('ES', '202406', 'CME')

In [7]:
await ib.qualifyContractsAsync(contract)

[Future(conId=551601561, symbol='ES', lastTradeDateOrContractMonth='20240621', multiplier='50', exchange='CME', currency='USD', localSymbol='ESM4', tradingClass='ES')]

In [8]:
from ib_insync import *

# Assume 'ib' is already connected and is an instance of ib_insync.IB()
# Also assume 'contract' has been created and is a valid ib_insync.Contract object

# Use the asynchronous version of the method with await
try:
    historical_data = await ib.reqHistoricalDataAsync(
        contract, endDateTime='', durationStr='30 D',
        barSizeSetting='1 hour', whatToShow='MIDPOINT', useRTH=True
    )
    
    # Proceed with the historical data
#     for bar in historical_data:
#         print(bar)
except Exception as e:
    print(f"An error occurred: {e}")


In [9]:
import pytz
import pandas as pd

# Assuming 'historical_data' contains the historical data returned by reqHistoricalDataAsync

# Convert timestamps to the desired time zone (e.g., 'US/Eastern')
tz = pytz.timezone('US/Eastern')
for bar in historical_data:
    bar.date = bar.date.astimezone(tz)

# Convert historical data to DataFrame
df = pd.DataFrame([vars(bar) for bar in historical_data])

# Print the DataFrame
print(df)


                         date     open     high      low    close  volume  \
0   2024-03-18 09:30:00-04:00  5225.75  5233.75  5222.50  5223.75    -1.0   
1   2024-03-18 10:00:00-04:00  5223.75  5240.25  5223.00  5237.00    -1.0   
2   2024-03-18 11:00:00-04:00  5237.00  5237.75  5224.00  5227.50    -1.0   
3   2024-03-18 12:00:00-04:00  5227.50  5230.50  5219.50  5220.25    -1.0   
4   2024-03-18 13:00:00-04:00  5220.25  5227.50  5217.75  5223.00    -1.0   
..                        ...      ...      ...      ...      ...     ...   
235 2024-04-29 12:00:00-04:00  5143.75  5150.00  5142.00  5146.25    -1.0   
236 2024-04-29 13:00:00-04:00  5146.25  5152.00  5142.50  5148.75    -1.0   
237 2024-04-29 14:00:00-04:00  5148.75  5148.75  5132.25  5142.25    -1.0   
238 2024-04-29 15:00:00-04:00  5142.25  5148.75  5118.50  5147.25    -1.0   
239 2024-04-29 16:00:00-04:00  5147.25  5151.75  5146.50  5147.50    -1.0   

     average  barCount  
0       -1.0        -1  
1       -1.0        -1  


In [10]:
df = util.df(historical_data)
print(df)
def preprocess_and_save_data(data, filename):
    data.to_csv(filename, index=False)
preprocess_and_save_data(df, 'ib_es500_price_history.csv')

                         date     open     high      low    close  volume  \
0   2024-03-18 09:30:00-04:00  5225.75  5233.75  5222.50  5223.75    -1.0   
1   2024-03-18 10:00:00-04:00  5223.75  5240.25  5223.00  5237.00    -1.0   
2   2024-03-18 11:00:00-04:00  5237.00  5237.75  5224.00  5227.50    -1.0   
3   2024-03-18 12:00:00-04:00  5227.50  5230.50  5219.50  5220.25    -1.0   
4   2024-03-18 13:00:00-04:00  5220.25  5227.50  5217.75  5223.00    -1.0   
..                        ...      ...      ...      ...      ...     ...   
235 2024-04-29 12:00:00-04:00  5143.75  5150.00  5142.00  5146.25    -1.0   
236 2024-04-29 13:00:00-04:00  5146.25  5152.00  5142.50  5148.75    -1.0   
237 2024-04-29 14:00:00-04:00  5148.75  5148.75  5132.25  5142.25    -1.0   
238 2024-04-29 15:00:00-04:00  5142.25  5148.75  5118.50  5147.25    -1.0   
239 2024-04-29 16:00:00-04:00  5147.25  5151.75  5146.50  5147.50    -1.0   

     average  barCount  
0       -1.0        -1  
1       -1.0        -1  


In [15]:
!pip install nest_asyncio

In [19]:
import nest_asyncio
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
nest_asyncio.apply()
# Load and preprocess data
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    # Adding more features: high, low, and volume
    features = data[['close', 'high', 'low', 'volume']].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_features = scaler.fit_transform(features)
    return scaled_features, scaler

# Create sequences for LSTM training
def create_sequences(data, sequence_length=60):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length, 0]  # Predicting the next close price
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Define LSTM model structure
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Adjust the prediction function to handle new features
def predict_and_decide(model, scaler, recent_data, current_price, available_funds, current_shares, total_cost, transaction_fee=10.0):
#     if recent_data.shape[0] != 60:
#         raise ValueError("recent_data should contain exactly 60 timesteps")
    
#     # Scale the recent_data if it's not already scaled
#     scaled_recent_data = scaler.transform(recent_data)
#     scaled_recent_data = scaled_recent_data.reshape(1, 60, -1)  # Reshape for LSTM input
#     prediction = model.predict(scaled_recent_data)
#     predicted_price = scaler.inverse_transform(prediction)[0][0]
    # Assuming recent_data is in the correct shape and already scaled appropriately
    scaled_recent_data = recent_data.reshape(1, 60, -1)  # Reshape for LSTM input
    
    prediction_scaled = model.predict(scaled_recent_data)
    
    # Since we're only interested in the 'close' price (first feature), we need to prepare a dummy array for inverse transform
    dummy_features = np.zeros((1, 4))  # Assume 4 features as per your scaler fitting
    dummy_features[0, 0] = prediction_scaled[0][0]  # Place the predicted price in the 'close' position
    
    predicted_price = scaler.inverse_transform(dummy_features)[0, 0]  # Inverse transform and extract the 'close' price
    print(predicted_price, current_price)
    # Decision making with transaction fees considered
    decision = "HOLD"
    shares = 0
    if predicted_price > current_price:
        if available_funds > transaction_fee:
            max_affordable_shares = int((available_funds - transaction_fee) / current_price)
            decision = "BUY"
            shares = max_affordable_shares  # Example: Buy as much as possible within budget
    elif predicted_price < current_price:
        decision = "SELL"
        shares = int(current_shares * 0.1)  # Example: Sell 10% of holdings
    
    return decision, shares, predicted_price

# Example usage of the functions
if __name__ == "__main__":
    filepath = 'ib_es500_price_history.csv'
    scaled_features, scaler = load_and_preprocess_data(filepath)
    X, y = create_sequences(scaled_features)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = build_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))  # Reduced epochs, added explicit test data for validation
    
#     current_shares = 0 #
    # Simulating a scenario for prediction and decision making
    positions = ib.positions()
    for pos in positions:
        if pos.contract.symbol == 'ES' and pos.contract.secType == 'FUT':
            current_shares = pos.position
    account_summary = await ib.accountSummaryAsync()
    # Find the total cash balance
    balance = next((item for item in account_summary if item.tag == 'TotalCashBalance'), None)
    
    contract = Future('ES', '202406', 'CME')
    market_data = ib.reqMktData(contract, '', False, False)
    util.sleep(5)
    current_price = (market_data.bid + market_data.ask) / 2
    
    
    available_funds = balance.value
    total_cost = current_shares * current_price  # Simulated total cost of current shares
    
    # Prepare recent data for prediction
    recent_data = scaled_features[-60:]  # Make sure this is exactly 60 timesteps
    decision, shares, predicted_price = predict_and_decide(model, scaler, recent_data, current_price, available_funds, current_shares, total_cost)
    print(f"Decision: {decision}, Shares: {shares}, Predicted Price: {predicted_price}")
    
    # Execute the decision
    if decision == "BUY":
        order = MarketOrder('BUY', shares)
        trade = ib.placeOrder(contract, order)
    elif decision == "SELL":
        order = MarketOrder('SELL', shares)
        trade = ib.placeOrder(contract, order)
    elif decision == "HOLD":
        print("Decision is to HOLD, no action taken.")

    # Monitor the order until it is filled
    ib.sleep(1)  # Sleeping to give time for the order to be executed


    print(f"Order {decision} for {shares} shares has been filled.")


2024-04-29 18:14:10.172467: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 18:14:10.174093: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 18:14:10.175426: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/50


2024-04-29 18:14:10.406139: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 18:14:10.407716: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 18:14:10.408996: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3/5 [=================>............] - ETA: 0s - loss: 0.1896 

2024-04-29 18:14:13.777384: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 18:14:13.778608: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 18:14:13.779776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5/5 [==============================] - 4s 179ms/step - loss: 0.1369 - val_loss: 0.0485
Epoch 2/50
5/5 [==============================] - 0s 31ms/step - loss: 0.0568 - val_loss: 0.0125
Epoch 3/50
5/5 [==============================] - 0s 33ms/step - loss: 0.0270 - val_loss: 0.0326
Epoch 4/50
5/5 [==============================] - 0s 32ms/step - loss: 0.0308 - val_loss: 0.0129
Epoch 5/50
5/5 [==============================] - 0s 31ms/step - loss: 0.0212 - val_loss: 0.0186
Epoch 6/50
5/5 [==============================] - 0s 32ms/step - loss: 0.0249 - val_loss: 0.0112
Epoch 7/50
5/5 [==============================] - 0s 32ms/step - loss: 0.0185 - val_loss: 0.0140
Epoch 8/50
5/5 [==============================] - 0s 33ms/step - loss: 0.0205 - val_loss: 0.0107
Epoch 9/50
5/5 [==============================] - 0s 33ms/step - loss: 0.0146 - val_loss: 0.0114
Epoch 10/50
5/5 [==============================] - 0s 32ms/step - loss: 0.0173 - val_loss: 0.0098
Epoch 11/50
5/5 [======================

Error 354, reqId 14: Requested market data is not subscribed.Delayed market data is available.ES JUN'24/TOP/ALL, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202406', exchange='CME')
2024-04-29 18:14:27.218113: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 18:14:27.220160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 18:14:27.221647

1/1 [==============================] - 1s 627ms/step
5135.1751851663 nan
Decision: HOLD, Shares: 0, Predicted Price: 5135.1751851663
Decision is to HOLD, no action taken.
Order HOLD for 0 shares has been filled.
